# 🏓 Table Tennis Match Prediction - Gold + EDA (Final Best)

**Version**: Gold + EDA - Optimized with Exploratory Data Analysis

**Performance**: 
- Private Score: 0.3596 ✅ **BEST**
- Public Score: 0.3189
- Delta: +0.0407 (excellent generalization)

**Key Improvements over Gold Baseline**:
1. ✅ Early rally features (first 3-4 strokes critical)
2. ✅ Hand-spin combinations (tactical patterns)
3. ✅ Improved lag features (previous 2 strokes)
4. ✅ Score momentum features
5. ✅ EDA-driven feature selection

**Comparison**:
- vs V6: +177 points (17.7% improvement)
- vs Gold: +22 points (2.2% improvement)

**Note**: This is the final competition version with the best private score.

## 1. Configuration & Setup

In [ ]:
# =========================================================
# 🔧 Configuration & Setup
# =========================================================

USE_GPU = True
N_FOLDS = 5
RANDOM_SEED = 42

# Install packages
print("Installing packages...")
# !pip -q install lightgbm catboost pandas numpy scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, roc_auc_score
import warnings
import sys

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"✓ Setup complete")
print(f"  Python: {sys.version.split(' ')[0]}")
print(f"  Random Seed: {RANDOM_SEED}")

## 2. Load Data

In [ ]:
print("\n[2/15] Loading data...")

try:
    # Try loading from parent data/ directory (when running from notebooks/)
    train_df = pd.read_csv("../data/train.csv")
    test_df = pd.read_csv("../data/test.csv")
    submission_df = pd.read_csv("../data/sample_submission.csv")
    print(f"✓ Data loaded from ../data/ directory")
except FileNotFoundError:
    try:
        # Try loading from local data/ directory
        train_df = pd.read_csv("data/train.csv")
        test_df = pd.read_csv("data/test.csv")
        submission_df = pd.read_csv("data/sample_submission.csv")
        print(f"✓ Data loaded from data/ directory")
    except FileNotFoundError:
        print("❌ Error: Data files not found")
        sys.exit(1)

print(f"\n  Train: {train_df.shape}")
print(f"  Test:  {test_df.shape}")
print(f"  Submission: {submission_df.shape}")

---

# 📊 PART 1: Exploratory Data Analysis (EDA)

Run all cells in this section and review the outputs before proceeding.

In [ ]:
# =========================================================
# EDA 1: Basic Statistics
# =========================================================

print("="*60)
print("EDA 1: BASIC STATISTICS")
print("="*60)

print("\n1. Dataset Overview:")
print(f"  Train rows: {len(train_df):,}")
print(f"  Test rows:  {len(test_df):,}")
print(f"  Train rallies: {train_df['rally_uid'].nunique():,}")
print(f"  Test rallies:  {test_df['rally_uid'].nunique():,}")

print("\n2. Rally Length Statistics:")
train_rally_len = train_df.groupby('rally_uid').size()
test_rally_len = test_df.groupby('rally_uid').size()

print("  Train Rally Length:")
print(f"    Mean: {train_rally_len.mean():.2f}")
print(f"    Median: {train_rally_len.median():.1f}")
print(f"    Min: {train_rally_len.min()}")
print(f"    Max: {train_rally_len.max()}")

print("\n  Test Rally Length:")
print(f"    Mean: {test_rally_len.mean():.2f}")
print(f"    Median: {test_rally_len.median():.1f}")
print(f"    Min: {test_rally_len.min()}")
print(f"    Max: {test_rally_len.max()}")

print("\n3. Missing Values:")
print("  Train:")
train_missing = train_df.isnull().sum()
print(train_missing[train_missing > 0] if train_missing.sum() > 0 else "    None")

print("\n  Test:")
test_missing = test_df.isnull().sum()
print(test_missing[test_missing > 0] if test_missing.sum() > 0 else "    None")

print("\n4. Target Variable (serverGetPoint) in Train:")
print(f"  Server Win Rate: {train_df['serverGetPoint'].mean():.2%}")
print(f"  Distribution:")
print(train_df['serverGetPoint'].value_counts(normalize=True).to_string())

## 3. EDA - Basic Statistics

In [ ]:
# =========================================================
# EDA 2: Feature Distribution Comparison
# =========================================================

print("\n" + "="*60)
print("EDA 2: FEATURE DISTRIBUTION (Train vs Test)")
print("="*60)

def compare_distributions(feature, train, test):
    """Compare distribution of a feature between train and test."""
    print(f"\n{feature}:")
    
    # Value counts
    train_dist = train[feature].value_counts(normalize=True).sort_index()
    test_dist = test[feature].value_counts(normalize=True).sort_index()
    
    # Common values
    common_vals = set(train_dist.index) & set(test_dist.index)
    if len(common_vals) > 0:
        diff = []
        for val in sorted(common_vals)[:10]:  # Show top 10
            train_pct = train_dist.get(val, 0) * 100
            test_pct = test_dist.get(val, 0) * 100
            diff_pct = test_pct - train_pct
            print(f"  {val:>4}: Train={train_pct:5.2f}%, Test={test_pct:5.2f}%, Diff={diff_pct:+5.2f}%")
    
    # Unique values
    print(f"  Unique in Train: {len(train_dist)}")
    print(f"  Unique in Test:  {len(test_dist)}")
    train_only = set(train_dist.index) - set(test_dist.index)
    test_only = set(test_dist.index) - set(train_dist.index)
    if train_only:
        print(f"  Only in Train: {len(train_only)} values")
    if test_only:
        print(f"  Only in Test: {len(test_only)} values")

# Compare key features
features_to_compare = ['actionId', 'pointId', 'handId', 'spinId', 'strengthId', 'strickId']

for feature in features_to_compare:
    if feature in train_df.columns and feature in test_df.columns:
        compare_distributions(feature, train_df, test_df)

## 4. EDA - Feature Distributions

In [ ]:
# =========================================================
# EDA 3: Score Distribution Analysis
# =========================================================

print("\n" + "="*60)
print("EDA 3: SCORE DISTRIBUTION")
print("="*60)

if 'scoreSelf' in train_df.columns and 'scoreOther' in train_df.columns:
    print("\n1. Score Statistics:")
    print("  Train:")
    print(f"    scoreSelf:  mean={train_df['scoreSelf'].mean():.2f}, max={train_df['scoreSelf'].max()}")
    print(f"    scoreOther: mean={train_df['scoreOther'].mean():.2f}, max={train_df['scoreOther'].max()}")
    
    if 'scoreSelf' in test_df.columns:
        print("  Test:")
        print(f"    scoreSelf:  mean={test_df['scoreSelf'].mean():.2f}, max={test_df['scoreSelf'].max()}")
        print(f"    scoreOther: mean={test_df['scoreOther'].mean():.2f}, max={test_df['scoreOther'].max()}")
    
    # Score difference
    train_df['score_diff_temp'] = train_df['scoreSelf'] - train_df['scoreOther']
    print("\n2. Score Difference (scoreSelf - scoreOther):")
    print(f"  Train: mean={train_df['score_diff_temp'].mean():.2f}, std={train_df['score_diff_temp'].std():.2f}")
    
    # Critical points (score >= 9)
    train_critical = ((train_df['scoreSelf'] >= 9) | (train_df['scoreOther'] >= 9)).sum()
    print(f"\n3. Critical Points (either side >= 9):")
    print(f"  Train: {train_critical:,} ({train_critical/len(train_df)*100:.2f}%)")
    
    if 'scoreSelf' in test_df.columns:
        test_df['score_diff_temp'] = test_df['scoreSelf'] - test_df['scoreOther']
        print(f"  Test: mean={test_df['score_diff_temp'].mean():.2f}, std={test_df['score_diff_temp'].std():.2f}")
        test_critical = ((test_df['scoreSelf'] >= 9) | (test_df['scoreOther'] >= 9)).sum()
        print(f"  Test: {test_critical:,} ({test_critical/len(test_df)*100:.2f}%)")
else:
    print("  Score columns not found in dataset")

## 5. EDA - Score Analysis

In [ ]:
# =========================================================
# EDA 4: Action Type Analysis
# =========================================================

print("\n" + "="*60)
print("EDA 4: ACTION TYPE PATTERNS")
print("="*60)

# Create action type mapping
action_map = {
    1: 'Attack', 2: 'Attack', 3: 'Attack', 4: 'Attack', 5: 'Attack', 6: 'Attack', 7: 'Attack',
    8: 'Control', 9: 'Control', 10: 'Control', 11: 'Control',
    12: 'Defensive', 13: 'Defensive', 14: 'Defensive',
    15: 'Serve', 16: 'Serve', 17: 'Serve', 18: 'Serve',
    0: 'Zero', -1: 'End'
}

train_df['action_type_temp'] = train_df['actionId'].map(action_map).fillna('Unknown')
test_df['action_type_temp'] = test_df['actionId'].map(action_map).fillna('Unknown')

print("\n1. Action Type Distribution:")
print("\n  Train:")
train_action_dist = train_df['action_type_temp'].value_counts(normalize=True)
for action, pct in train_action_dist.items():
    print(f"    {action:12s}: {pct*100:5.2f}%")

print("\n  Test:")
test_action_dist = test_df['action_type_temp'].value_counts(normalize=True)
for action, pct in test_action_dist.items():
    print(f"    {action:12s}: {pct*100:5.2f}%")

# Win rate by action type
if 'serverGetPoint' in train_df.columns:
    print("\n2. Server Win Rate by Action Type:")
    win_by_action = train_df.groupby('action_type_temp')['serverGetPoint'].mean()
    for action, rate in win_by_action.sort_values(ascending=False).items():
        print(f"    {action:12s}: {rate*100:5.2f}%")

## 6. EDA - Action Types

In [ ]:
# =========================================================
# EDA 5: Rally Phase Analysis
# =========================================================

print("\n" + "="*60)
print("EDA 5: RALLY PHASE PATTERNS")
print("="*60)

def get_rally_phase(n):
    if n == 1: return 'Serve'
    elif n == 2: return 'Return'
    elif n <= 4: return 'Early'
    else: return 'Extended'

train_df['rally_phase_temp'] = train_df['strickNumber'].apply(get_rally_phase)
test_df['rally_phase_temp'] = test_df['strickNumber'].apply(get_rally_phase)

print("\n1. Stroke Distribution by Rally Phase:")
print("\n  Train:")
train_phase_dist = train_df['rally_phase_temp'].value_counts(normalize=True)
for phase, pct in train_phase_dist.items():
    print(f"    {phase:12s}: {pct*100:5.2f}%")

print("\n  Test:")
test_phase_dist = test_df['rally_phase_temp'].value_counts(normalize=True)
for phase, pct in test_phase_dist.items():
    print(f"    {phase:12s}: {pct*100:5.2f}%")

# Win rate by rally phase
if 'serverGetPoint' in train_df.columns:
    print("\n2. Server Win Rate by Rally Phase:")
    win_by_phase = train_df.groupby('rally_phase_temp')['serverGetPoint'].mean()
    for phase, rate in win_by_phase.items():
        print(f"    {phase:12s}: {rate*100:5.2f}%")

# Action type by rally phase
print("\n3. Top Action Types by Rally Phase (Train):")
for phase in ['Serve', 'Return', 'Early', 'Extended']:
    if phase in train_df['rally_phase_temp'].values:
        print(f"\n  {phase}:")
        phase_actions = train_df[train_df['rally_phase_temp'] == phase]['action_type_temp'].value_counts(normalize=True).head(3)
        for action, pct in phase_actions.items():
            print(f"    {action:12s}: {pct*100:5.2f}%")

## 7. EDA - Rally Phases

In [ ]:
# =========================================================
# EDA 6: Visualizations
# =========================================================

print("\n" + "="*60)
print("EDA 6: VISUALIZATIONS")
print("="*60)

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Train vs Test Distribution Comparison', fontsize=16, fontweight='bold')

# 1. Rally Length Distribution
ax = axes[0, 0]
train_rally_len.value_counts().sort_index().head(20).plot(kind='bar', ax=ax, alpha=0.7, label='Train')
test_rally_len.value_counts().sort_index().head(20).plot(kind='bar', ax=ax, alpha=0.7, label='Test')
ax.set_title('Rally Length Distribution')
ax.set_xlabel('Number of Strokes')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Action Type Distribution
ax = axes[0, 1]
# Get all unique action types from both train and test
all_action_types = sorted(set(train_action_dist.index) | set(test_action_dist.index))
x = np.arange(len(all_action_types))
width = 0.35
train_vals = [train_action_dist.get(action, 0) * 100 for action in all_action_types]
test_vals = [test_action_dist.get(action, 0) * 100 for action in all_action_types]
ax.bar(x - width/2, train_vals, width, label='Train', alpha=0.7)
ax.bar(x + width/2, test_vals, width, label='Test', alpha=0.7)
ax.set_title('Action Type Distribution')
ax.set_ylabel('Percentage (%)')
ax.set_xticks(x)
ax.set_xticklabels(all_action_types, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Rally Phase Distribution
ax = axes[1, 0]
phase_order = ['Serve', 'Return', 'Early', 'Extended']
x = np.arange(len(phase_order))
train_vals = [train_phase_dist.get(p, 0) * 100 for p in phase_order]
test_vals = [test_phase_dist.get(p, 0) * 100 for p in phase_order]
ax.bar(x - width/2, train_vals, width, label='Train', alpha=0.7)
ax.bar(x + width/2, test_vals, width, label='Test', alpha=0.7)
ax.set_title('Rally Phase Distribution')
ax.set_ylabel('Percentage (%)')
ax.set_xticks(x)
ax.set_xticklabels(phase_order)
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Score Distribution (if available)
ax = axes[1, 1]
if 'scoreSelf' in train_df.columns and 'scoreSelf' in test_df.columns:
    train_df['scoreSelf'].value_counts().sort_index().plot(kind='line', ax=ax, label='Train', marker='o')
    test_df['scoreSelf'].value_counts().sort_index().plot(kind='line', ax=ax, label='Test', marker='s')
    ax.set_title('Score Distribution (scoreSelf)')
    ax.set_xlabel('Score')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True, alpha=0.3)
else:
    ax.text(0.5, 0.5, 'Score data not available', ha='center', va='center', transform=ax.transAxes)
    ax.set_title('Score Distribution')

# 5. Server Win Rate by Action Type (Train only)
ax = axes[2, 0]
if 'serverGetPoint' in train_df.columns:
    win_by_action.sort_values().plot(kind='barh', ax=ax, color='skyblue', alpha=0.7)
    ax.set_title('Server Win Rate by Action Type (Train)')
    ax.set_xlabel('Win Rate')
    ax.axvline(0.5, color='red', linestyle='--', linewidth=1, alpha=0.5)
    ax.grid(True, alpha=0.3)
else:
    ax.text(0.5, 0.5, 'Target variable not available', ha='center', va='center', transform=ax.transAxes)
    ax.set_title('Server Win Rate by Action Type')

# 6. actionId Distribution (Top 15)
ax = axes[2, 1]
train_action_id = train_df['actionId'].value_counts().head(15)
test_action_id = test_df['actionId'].value_counts().head(15)
all_actions = sorted(set(train_action_id.index) | set(test_action_id.index))
x = np.arange(len(all_actions))
train_vals = [train_action_id.get(a, 0) for a in all_actions]
test_vals = [test_action_id.get(a, 0) for a in all_actions]
ax.bar(x - width/2, train_vals, width, label='Train', alpha=0.7)
ax.bar(x + width/2, test_vals, width, label='Test', alpha=0.7)
ax.set_title('ActionId Distribution (Top 15)')
ax.set_ylabel('Frequency')
ax.set_xticks(x)
ax.set_xticklabels(all_actions, rotation=45)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../analysis/eda_analysis.png', dpi=300, bbox_inches='tight')
print("\n✓ Visualization saved to '../analysis/eda_analysis.png'")
plt.show()

## 8. EDA - Visualizations

---

# 🔨 PART 2: Feature Engineering (Based on EDA)

**⚠️ DO NOT RUN YET - Wait for EDA-based adjustments**

In [ ]:
# =========================================================
# 🔨 PART 2: Feature Engineering (EDA-Optimized)
# =========================================================

print("\n[3/8] Feature Engineering (EDA-Optimized)...")

def get_rally_phase(n):
    """Categorize rally by stroke number."""
    if n == 1: return 0      # Serve
    elif n == 2: return 1    # Return
    elif n <= 4: return 2    # Early
    else: return 3           # Extended

def create_features_eda(df):
    """
    EDA-Optimized Feature Engineering
    
    Key insights from EDA:
    1. Test has MUCH shorter rallies (median=2 vs train=5)
    2. Test has MORE serves (32.7% vs 15.9%)
    3. handId=1 is dominant in test (61% vs 45%)
    4. spinId=2 and strengthId=2/3 more common in test
    5. Early phases more important for test set
    """
    df_feats = df.copy()
    
    # === 1. Basic Features ===
    df_feats['rally_phase'] = df_feats['strickNumber'].apply(get_rally_phase)
    
    action_map = {
        1: 'Attack', 2: 'Attack', 3: 'Attack', 4: 'Attack', 5: 'Attack', 6: 'Attack', 7: 'Attack',
        8: 'Control', 9: 'Control', 10: 'Control', 11: 'Control',
        12: 'Defensive', 13: 'Defensive', 14: 'Defensive',
        15: 'Serve', 16: 'Serve', 17: 'Serve', 18: 'Serve',
        0: 'Zero', -1: 'End'
    }
    df_feats['action_type'] = df_feats['actionId'].map(action_map).fillna('Zero')
    
    # Score features
    if 'scoreSelf' in df_feats.columns and 'scoreOther' in df_feats.columns:
        df_feats['score_diff'] = df_feats['scoreSelf'] - df_feats['scoreOther']
        df_feats['score_sum'] = df_feats['scoreSelf'] + df_feats['scoreOther']
        # Critical point indicator (21% of data)
        df_feats['is_critical'] = ((df_feats['scoreSelf'] >= 9) | (df_feats['scoreOther'] >= 9)).astype(int)
    
    # === 2. EDA Insight: Early-phase specific features ===
    # Since test has more early strokes, focus on serve/return characteristics
    df_feats['is_serve_phase'] = (df_feats['strickNumber'] == 1).astype(int)
    df_feats['is_return_phase'] = (df_feats['strickNumber'] == 2).astype(int)
    df_feats['is_early'] = (df_feats['strickNumber'] <= 4).astype(int)
    
    # === 3. Lag-1 Features (most important for short rallies) ===
    lag1_cols = ['strickId', 'handId', 'strengthId', 'spinId', 'pointId', 
                 'actionId', 'positionId', 'action_type']
    for col in lag1_cols:
        df_feats[f'prev_{col}'] = df_feats.groupby('rally_uid')[col].shift(1)
    
    # === 4. Conditional Lag-2 (only useful for longer rallies) ===
    # Don't over-rely on Lag-2 since test rallies are shorter
    lag2_cols = ['actionId', 'pointId']
    for col in lag2_cols:
        df_feats[f'prev2_{col}'] = df_feats.groupby('rally_uid')[col].shift(2)
    
    # === 5. Tactical Combinations ===
    df_feats['prev_hand_spin'] = (
        df_feats['prev_handId'].astype(str) + '_' + df_feats['prev_spinId'].astype(str)
    )
    df_feats['prev_action_point'] = (
        df_feats['prev_actionId'].astype(str) + '_' + df_feats['prev_pointId'].astype(str)
    )
    df_feats['prev_action_phase'] = (
        df_feats['prev_actionId'].astype(str) + '_ph' + df_feats['rally_phase'].astype(str)
    )
    
    # === 6. EDA-Driven: Hand-Spin-Strength combinations ===
    # handId=1, spinId=2, strengthId=2/3 are more common in test
    df_feats['hand_spin'] = (
        df_feats['handId'].astype(str) + '_' + df_feats['spinId'].astype(str)
    )
    df_feats['hand_strength'] = (
        df_feats['handId'].astype(str) + '_' + df_feats['strengthId'].astype(str)
    )
    
    # === 7. Action change indicator (rally dynamics) ===
    df_feats['action_changed'] = (df_feats['actionId'] != df_feats['prev_actionId']).astype(int)
    
    # === 8. Fill Missing Values ===
    for col in df_feats.columns:
        if 'prev' in col:
            df_feats[col] = df_feats[col].replace(
                ['nan_nan', 'nan', '<NA>', '<NA>_<NA>'], np.nan
            )
        
        if col.startswith('prev') or col in ['hand_spin', 'hand_strength']:
            if df_feats[col].dtype == 'object':
                df_feats[col] = df_feats[col].fillna('None')
            else:
                df_feats[col] = df_feats[col].fillna(-999)
    
    return df_feats

# Create features
print("  Creating EDA-optimized features...")
train_feats_df = create_features_eda(train_df)
test_feats_df = create_features_eda(test_df)

print(f"✓ Feature engineering complete")
print(f"  Train shape: {train_feats_df.shape}")
print(f"  Test shape: {test_feats_df.shape}")

## 9. Feature Engineering (EDA-Optimized)

---

# 🤖 PART 3: Model Training (Optimized)

**⚠️ DO NOT RUN YET - Wait for EDA-based adjustments**

In [ ]:
# =========================================================
# 🤖 PART 3: Data Preparation & Model Training
# =========================================================

print("\n[4/8] Preparing Datasets...")

# Create target variables
train_feats_df['next_actionId'] = train_feats_df.groupby('rally_uid')['actionId'].shift(-1)
train_feats_df['next_pointId'] = train_feats_df.groupby('rally_uid')['pointId'].shift(-1)
train_feats_df['rally_outcome'] = train_feats_df['serverGetPoint']

# Filter rows with valid next actions
train_next_df = train_feats_df.dropna(subset=['next_actionId', 'next_pointId']).copy()

# Define columns to drop
drop_cols = [
    'rally_uid', 'serverGetPoint', 'gamePlayerId', 'gamePlayerOtherId',
    'match_id', 'next_actionId', 'next_pointId', 'rally_outcome', 'match', 'rally_id',
    # Drop temporary columns from EDA
    'action_type_temp', 'rally_phase_temp', 'score_diff_temp'
]
features = [col for col in train_feats_df.columns if col not in drop_cols and col in test_feats_df.columns]

# Identify categorical features
categorical_features = []
for col in features:
    if train_feats_df[col].dtype == 'object' or 'Id' in col or 'phase' in col or col.startswith('is_'):
        categorical_features.append(col)

# Encode categorical features
print(f"  Encoding {len(categorical_features)} categorical features...")
for col in categorical_features:
    le = LabelEncoder()
    train_feats_df[col] = train_feats_df[col].astype(str)
    test_feats_df[col] = test_feats_df[col].astype(str)
    le.fit(pd.concat([train_feats_df[col], test_feats_df[col]]))
    train_feats_df[col] = le.transform(train_feats_df[col])
    test_feats_df[col] = le.transform(test_feats_df[col])

# Prepare datasets
X_next = train_feats_df.loc[train_next_df.index, features]
groups_next = train_next_df['rally_uid']

le_action = LabelEncoder()
y_action = le_action.fit_transform(train_next_df['next_actionId'].astype(int))

le_point = LabelEncoder()
y_point = le_point.fit_transform(train_next_df['next_pointId'].astype(int))

X_outcome = train_feats_df[features]
y_outcome = train_feats_df['rally_outcome']
groups_outcome = train_feats_df['rally_uid']

test_final_rows = test_feats_df.groupby('rally_uid').tail(1)
X_test = test_final_rows[features]
test_rally_uids = test_final_rows['rally_uid']

print(f"✓ Data preparation complete")
print(f"  Features: {len(features)}")
print(f"  X_next: {X_next.shape}")
print(f"  X_outcome: {X_outcome.shape}")
print(f"  X_test: {X_test.shape}")

# =========================================================
# Model Training Functions
# =========================================================

def train_lgb(X, y, groups, X_test, params, cat_feats, n_splits=5):
    """Train LightGBM with GroupKFold cross-validation."""
    gkf = GroupKFold(n_splits=n_splits)
    num_class = params.get('num_class', 1)
    is_multiclass = params['objective'] == 'multiclass'
    
    oof_preds = np.zeros((len(X), num_class)) if is_multiclass else np.zeros(len(X))
    test_preds_list = []
    
    for train_idx, val_idx in gkf.split(X, y, groups):
        X_tr, y_tr = X.iloc[train_idx], y[train_idx]
        X_val, y_val = X.iloc[val_idx], y[val_idx]
        
        # Handle unseen labels
        if is_multiclass:
            missing = set(np.unique(y_val)) - set(np.unique(y_tr))
            if missing:
                add_idx = [val_idx[np.where(y_val == label)[0][0]] for label in missing]
                X_tr = pd.concat([X_tr, X.iloc[add_idx]])
                y_tr = np.concatenate([y_tr, y[add_idx]])
        
        dtrain = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_feats)
        dval = lgb.Dataset(X_val, label=y_val, categorical_feature=cat_feats, reference=dtrain)
        
        model = lgb.train(
            params, dtrain, valid_sets=[dval],
            callbacks=[lgb.early_stopping(50, verbose=False), lgb.log_evaluation(0)]
        )
        
        oof_preds[val_idx] = model.predict(X.iloc[val_idx])
        test_preds_list.append(model.predict(X_test))
    
    return oof_preds, np.mean(test_preds_list, axis=0)


def train_cat(X, y, groups, X_test, params, cat_indices, n_splits=5):
    """Train CatBoost with GroupKFold cross-validation."""
    gkf = GroupKFold(n_splits=n_splits)
    is_multiclass = 'MultiClass' in params.get('loss_function', '')
    num_class = int(np.max(y) + 1) if is_multiclass else 1
    
    oof_preds = np.zeros((len(X), num_class)) if is_multiclass else np.zeros(len(X))
    test_preds_list = []
    
    for train_idx, val_idx in gkf.split(X, y, groups):
        X_tr, y_tr = X.iloc[train_idx], y[train_idx]
        X_val, y_val = X.iloc[val_idx], y[val_idx]
        
        # Handle unseen labels
        if is_multiclass:
            missing = set(np.unique(y_val)) - set(np.unique(y_tr))
            if missing:
                add_idx = [val_idx[np.where(y_val == label)[0][0]] for label in missing]
                X_tr = pd.concat([X_tr, X.iloc[add_idx]])
                y_tr = np.concatenate([y_tr, y[add_idx]])
        
        train_pool = Pool(X_tr, y_tr, cat_features=cat_indices)
        val_pool = Pool(X_val, y_val, cat_features=cat_indices)
        
        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=0)
        
        if is_multiclass:
            oof_preds[val_idx] = model.predict_proba(val_pool)
            test_preds_list.append(model.predict_proba(X_test))
        else:
            oof_preds[val_idx] = model.predict_proba(val_pool)[:, 1]
            test_preds_list.append(model.predict_proba(X_test)[:, 1])
    
    return oof_preds, np.mean(test_preds_list, axis=0)

# =========================================================
# Dual-Engine Training with EDA-based parameters
# =========================================================

print("\n[5/8] Training Models (EDA-Optimized)...")
print("  🎯 Adjustments based on EDA:")
print("     - Lower learning rate for better generalization")
print("     - Stronger regularization (test is different)")
print("     - 50-50 blending for stability")

# EDA-Optimized: Lower learning rate, stronger regularization
lgb_common = {
    'boosting_type': 'gbdt',
    'n_estimators': 3000,
    'learning_rate': 0.02,  # Lower for better generalization
    'num_leaves': 24,       # Reduced complexity
    'subsample': 0.75,      # More aggressive sampling
    'colsample_bytree': 0.75,
    'min_child_samples': 50,  # Stronger regularization
    'random_state': RANDOM_SEED,
    'n_jobs': -1,
    'verbose': -1
}

cat_common = {
    'iterations': 2000,
    'learning_rate': 0.03,  # Lower for generalization
    'depth': 6,             # Reduced depth
    'l2_leaf_reg': 5,       # L2 regularization
    'random_seed': RANDOM_SEED,
    'verbose': 0
}

if USE_GPU:
    lgb_common['device'] = 'gpu'
    cat_common.update({'task_type': 'GPU', 'devices': '0'})

cat_indices = [X_next.columns.get_loc(col) for col in categorical_features]

# === Train Action ID ===
print("\n>> Action ID Predictor...")
lgb_p = {**lgb_common, 'objective': 'multiclass', 
         'num_class': len(le_action.classes_), 'metric': 'multi_logloss'}
cat_p = {**cat_common, 'loss_function': 'MultiClass', 'eval_metric': 'MultiClass'}

oof_lgb1, pred_lgb1 = train_lgb(X_next, y_action, groups_next, X_test, lgb_p, categorical_features)
oof_cat1, pred_cat1 = train_cat(X_next, y_action, groups_next, X_test, cat_p, cat_indices)

oof_blended_action = 0.5 * oof_lgb1 + 0.5 * oof_cat1
final_proba_action = 0.5 * pred_lgb1 + 0.5 * pred_cat1
action_f1 = f1_score(y_action, np.argmax(oof_blended_action, axis=1), average='macro')
print(f"   ✓ Action OOF F1: {action_f1:.4f}")

# === Train Point ID ===
print("\n>> Point ID Predictor...")
lgb_p['num_class'] = len(le_point.classes_)

oof_lgb2, pred_lgb2 = train_lgb(X_next, y_point, groups_next, X_test, lgb_p, categorical_features)
oof_cat2, pred_cat2 = train_cat(X_next, y_point, groups_next, X_test, cat_p, cat_indices)

oof_blended_point = 0.5 * oof_lgb2 + 0.5 * oof_cat2
final_proba_point = 0.5 * pred_lgb2 + 0.5 * pred_cat2
point_f1 = f1_score(y_point, np.argmax(oof_blended_point, axis=1), average='macro')
print(f"   ✓ Point OOF F1: {point_f1:.4f}")

# === Train Outcome ===
print("\n>> Rally Outcome Predictor...")
lgb_p_bin = {**lgb_common, 'objective': 'binary', 'metric': 'auc'}
cat_p_bin = {**cat_common, 'loss_function': 'Logloss', 'eval_metric': 'AUC'}

oof_lgb3, pred_lgb3 = train_lgb(X_outcome, y_outcome, groups_outcome, X_test, lgb_p_bin, categorical_features)
oof_cat3, pred_cat3 = train_cat(X_outcome, y_outcome, groups_outcome, X_test, cat_p_bin, cat_indices)

oof_blended_outcome = 0.5 * oof_lgb3 + 0.5 * oof_cat3
final_proba_outcome = 0.5 * pred_lgb3 + 0.5 * pred_cat3
outcome_auc = roc_auc_score(y_outcome, oof_blended_outcome)
print(f"   ✓ Outcome OOF AUC: {outcome_auc:.4f}")

print("\n✓ Training complete!")

## 10. Data Preparation & Model Training

In [ ]:
# =========================================================
# 🔄 Prediction Synchronization
# =========================================================

print("\n[6/8] Synchronizing Predictions...")

def synchronize_endings(prob_act, prob_pt, le_act, le_pt, threshold=0.5):
    """Synchronize action and point predictions for rally endings."""
    try:
        act_neg1_idx = list(le_act.classes_).index(-1)
        pt_neg1_idx = list(le_pt.classes_).index(-1)
        
        p_end = (prob_act[:, act_neg1_idx] + prob_pt[:, pt_neg1_idx]) / 2
        
        prob_act_mod, prob_pt_mod = prob_act.copy(), prob_pt.copy()
        prob_act_mod[p_end >= threshold, act_neg1_idx] = 2.0
        prob_pt_mod[p_end >= threshold, pt_neg1_idx] = 2.0
        prob_act_mod[p_end < threshold, act_neg1_idx] = 0.0
        prob_pt_mod[p_end < threshold, pt_neg1_idx] = 0.0
        
        synced_count = (p_end >= threshold).sum()
        print(f"  ✓ Synchronized {synced_count} endings")
        
        return (le_act.inverse_transform(np.argmax(prob_act_mod, axis=1)),
                le_pt.inverse_transform(np.argmax(prob_pt_mod, axis=1)))
    except ValueError:
        print("  ⚠ Warning: -1 label not found")
        return (le_act.inverse_transform(np.argmax(prob_act, axis=1)),
                le_pt.inverse_transform(np.argmax(prob_pt, axis=1)))

final_action, final_point = synchronize_endings(
    final_proba_action, final_proba_point, le_action, le_point
)

## 11. Synchronize Predictions

In [ ]:
# =========================================================
# 📤 Generate Submission
# =========================================================

print("\n[7/8] Generating Submission...")

# Create submission dataframe
submission = pd.DataFrame({
    'rally_uid': test_rally_uids,
    'serverGetPoint': final_proba_outcome,
    'pointId': final_point,
    'actionId': final_action
})

# Merge with submission template
final_submission = pd.merge(submission_df[['rally_uid']], submission, on='rally_uid', how='left')

# Fill missing values
final_submission.fillna({'serverGetPoint': 0.5}, inplace=True)
valid_action_mode = train_df['actionId'].mode()[0]
valid_point_mode = train_df['pointId'].mode()[0]
final_submission['actionId'] = final_submission['actionId'].fillna(valid_action_mode).astype(int)
final_submission['pointId'] = final_submission['pointId'].fillna(valid_point_mode).astype(int)

print(f"✓ Submission created: {final_submission.shape}")

## 12. Generate Submission

In [ ]:
# Save
output_file = '../submissions/submission_gold_eda.csv'
final_submission.to_csv(output_file, index=False)

print(f"  ✓ Saved to '{output_file}'")
print(f"\n  Summary:")
print(f"    Total predictions: {len(final_submission)}")
print(f"    Action -1 (end): {(final_submission['actionId'] == -1).sum()}")
print(f"    Point -1 (end): {(final_submission['pointId'] == -1).sum()}")
print(f"    Mean serverGetPoint: {final_submission['serverGetPoint'].mean():.4f}")

print("\n" + "="*60)
print("✅ EDA-OPTIMIZED TRAINING COMPLETE!")
print("="*60)
print(f"\n📊 OOF Performance:")
print(f"  Action F1:   {action_f1:.4f}")
print(f"  Point F1:    {point_f1:.4f}")
print(f"  Outcome AUC: {outcome_auc:.4f}")
print(f"\n🎯 EDA-Based Optimizations Applied:")
print(f"  ✓ Early-phase focused features (test has 32.7% serves vs 15.9% train)")
print(f"  ✓ Hand-spin-strength combinations (aligned with test distribution)")
print(f"  ✓ Reduced Lag-2 dependency (test rallies are shorter)")
print(f"  ✓ Stronger regularization for domain adaptation")
print(f"  ✓ 50-50 blending for stability")
print("="*60)